## Interactieve Phi 3 Mini 4K Instruct Chatbot met Whisper

### Introductie:
De Interactieve Phi 3 Mini 4K Instruct Chatbot is een hulpmiddel waarmee gebruikers kunnen communiceren met de Microsoft Phi 3 Mini 4K instruct-demo via tekst- of audio-invoer. De chatbot kan worden gebruikt voor verschillende taken, zoals vertalingen, weerupdates en het verzamelen van algemene informatie.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Maak je Huggingface-toegangstoken aan

Maak een nieuw token aan  
Geef een nieuwe naam op  
Selecteer schrijfrechten  
Kopieer het token en bewaar het op een veilige plek


De volgende Python-code voert twee hoofdtaken uit: het importeren van de `os`-module en het instellen van een omgevingsvariabele.

1. Importeren van de `os`-module:
   - De `os`-module in Python biedt een manier om te communiceren met het besturingssysteem. Hiermee kun je verschillende taken uitvoeren die gerelateerd zijn aan het besturingssysteem, zoals toegang krijgen tot omgevingsvariabelen, werken met bestanden en mappen, enzovoort.
   - In deze code wordt de `os`-module geïmporteerd met behulp van de `import`-instructie. Deze instructie maakt de functionaliteit van de `os`-module beschikbaar voor gebruik in het huidige Python-script.

2. Instellen van een omgevingsvariabele:
   - Een omgevingsvariabele is een waarde die toegankelijk is voor programma's die op het besturingssysteem draaien. Het is een manier om configuratie-instellingen of andere informatie op te slaan die door meerdere programma's kan worden gebruikt.
   - In deze code wordt een nieuwe omgevingsvariabele ingesteld met behulp van de `os.environ`-dictionary. De sleutel van de dictionary is `'HF_TOKEN'`, en de waarde wordt toegewezen vanuit de variabele `HUGGINGFACE_TOKEN`.
   - De variabele `HUGGINGFACE_TOKEN` wordt net boven dit codefragment gedefinieerd en krijgt een stringwaarde `"hf_**************"` toegewezen met behulp van de `#@param`-syntax. Deze syntax wordt vaak gebruikt in Jupyter-notebooks om gebruikersinvoer en parameterconfiguratie rechtstreeks in de notebookinterface mogelijk te maken.
   - Door de omgevingsvariabele `'HF_TOKEN'` in te stellen, kan deze worden benaderd door andere delen van het programma of andere programma's die op hetzelfde besturingssysteem draaien.

Samenvattend importeert deze code de `os`-module en stelt een omgevingsvariabele genaamd `'HF_TOKEN'` in met de waarde die is opgegeven in de variabele `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Deze codefragment definieert een functie genaamd clear_output, die wordt gebruikt om de uitvoer van de huidige cel in Jupyter Notebook of IPython te wissen. Laten we de code opsplitsen en de functionaliteit ervan begrijpen:

De functie clear_output neemt één parameter genaamd wait, die een booleaanse waarde is. Standaard is wait ingesteld op False. Deze parameter bepaalt of de functie moet wachten totdat nieuwe uitvoer beschikbaar is om de bestaande uitvoer te vervangen voordat deze wordt gewist.

De functie zelf wordt gebruikt om de uitvoer van de huidige cel te wissen. In Jupyter Notebook of IPython, wanneer een cel uitvoer genereert, zoals afgedrukte tekst of grafische plots, wordt die uitvoer onder de cel weergegeven. De clear_output-functie stelt je in staat om die uitvoer te wissen.

De implementatie van de functie wordt niet verstrekt in het codefragment, zoals aangegeven door de ellipsis (...). De ellipsis vertegenwoordigt een placeholder voor de daadwerkelijke code die het wissen van de uitvoer uitvoert. De implementatie van de functie kan het gebruik van de Jupyter Notebook- of IPython-API omvatten om de bestaande uitvoer uit de cel te verwijderen.

Over het algemeen biedt deze functie een handige manier om de uitvoer van de huidige cel in Jupyter Notebook of IPython te wissen, waardoor het eenvoudiger wordt om de weergegeven uitvoer te beheren en bij te werken tijdens interactieve coderingssessies.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Voer tekst-naar-spraak (TTS) uit met behulp van de Edge TTS-service. Laten we de relevante functie-implementaties één voor één doornemen:

1. `calculate_rate_string(input_value)`: Deze functie neemt een invoerwaarde en berekent de snelheidsstring voor de TTS-stem. De invoerwaarde vertegenwoordigt de gewenste snelheid van de spraak, waarbij een waarde van 1 de normale snelheid aangeeft. De functie berekent de snelheidsstring door 1 af te trekken van de invoerwaarde, deze te vermenigvuldigen met 100, en vervolgens het teken te bepalen op basis van of de invoerwaarde groter dan of gelijk aan 1 is. De functie retourneert de snelheidsstring in het formaat "{sign}{rate}".

2. `make_chunks(input_text, language)`: Deze functie neemt een invoertekst en een taal als parameters. Het splitst de invoertekst in stukken op basis van taal-specifieke regels. In deze implementatie, als de taal "Engels" is, splitst de functie de tekst bij elke punt (".") en verwijdert eventuele leidende of afsluitende spaties. Vervolgens voegt het een punt toe aan elk stuk en retourneert de gefilterde lijst van stukken.

3. `tts_file_name(text)`: Deze functie genereert een bestandsnaam voor het TTS-audiobestand op basis van de invoertekst. Het voert verschillende transformaties uit op de tekst: het verwijderen van een afsluitende punt (indien aanwezig), het omzetten van de tekst naar kleine letters, het verwijderen van leidende en afsluitende spaties, en het vervangen van spaties door onderstrepingstekens. Vervolgens wordt de tekst ingekort tot maximaal 25 tekens (indien langer) of gebruikt de volledige tekst als deze leeg is. Ten slotte genereert het een willekeurige string met behulp van de [`uuid`] module en combineert deze met de ingekorte tekst om de bestandsnaam te maken in het formaat "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Deze functie voegt meerdere audiobestanden samen tot één audiobestand. Het neemt een lijst van audiobestandslocaties en een uitvoerlocatie als parameters. De functie initialiseert een lege `AudioSegment`-object genaamd [`merged_audio`]. Vervolgens doorloopt het elke audiobestandslocatie, laadt het audiobestand met behulp van de methode `AudioSegment.from_file()` van de `pydub`-bibliotheek, en voegt het huidige audiobestand toe aan het [`merged_audio`] object. Ten slotte exporteert het de samengevoegde audio naar de opgegeven uitvoerlocatie in MP3-formaat.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Deze functie voert de TTS-operatie uit met behulp van de Edge TTS-service. Het neemt een lijst van tekststukken, de snelheid van de spraak, de stemnaam en de opslaglocatie als parameters. Als het aantal stukken groter is dan 1, maakt de functie een map aan voor het opslaan van de individuele audiobestanden van de stukken. Vervolgens doorloopt het elk stuk, construeert een Edge TTS-commando met behulp van de functie `calculate_rate_string()`, de stemnaam en de tekst van het stuk, en voert het commando uit met behulp van de functie `os.system()`. Als de uitvoering van het commando succesvol is, voegt het het pad van het gegenereerde audiobestand toe aan een lijst. Na het verwerken van alle stukken, voegt het de individuele audiobestanden samen met behulp van de functie `merge_audio_files()` en slaat de samengevoegde audio op in de opgegeven opslaglocatie. Als er slechts één stuk is, genereert het direct het Edge TTS-commando en slaat de audio op in de opslaglocatie. Ten slotte retourneert het de opslaglocatie van het gegenereerde audiobestand.

6. `random_audio_name_generate()`: Deze functie genereert een willekeurige audiobestandsnaam met behulp van de [`uuid`] module. Het genereert een willekeurige UUID, zet deze om naar een string, neemt de eerste 8 tekens, voegt de ".mp3"-extensie toe, en retourneert de willekeurige audiobestandsnaam.

7. `talk(input_text)`: Deze functie is het belangrijkste toegangspunt voor het uitvoeren van de TTS-operatie. Het neemt een invoertekst als parameter. Het controleert eerst de lengte van de invoertekst om te bepalen of het een lange zin is (groter dan of gelijk aan 600 tekens). Op basis van de lengte en de waarde van de variabele `translate_text_flag` bepaalt het de taal en genereert het de lijst van tekststukken met behulp van de functie `make_chunks()`. Vervolgens genereert het een opslaglocatie voor het audiobestand met behulp van de functie `random_audio_name_generate()`. Ten slotte roept het de functie `edge_free_tts()` aan om de TTS-operatie uit te voeren en retourneert het de opslaglocatie van het gegenereerde audiobestand.

Samenvattend werken deze functies samen om de invoertekst op te splitsen in stukken, een bestandsnaam te genereren voor het audiobestand, de TTS-operatie uit te voeren met behulp van de Edge TTS-service, en de individuele audiobestanden samen te voegen tot één audiobestand.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

De implementatie van twee functies: convert_to_text en run_text_prompt, evenals de declaratie van twee klassen: str en Audio.

De functie convert_to_text neemt een audio_path als invoer en transcribeert de audio naar tekst met behulp van een model genaamd whisper_model. De functie controleert eerst of de gpu-vlag is ingesteld op True. Als dat het geval is, wordt het whisper_model gebruikt met bepaalde parameters zoals word_timestamps=True, fp16=True, language='English', en task='translate'. Als de gpu-vlag False is, wordt het whisper_model gebruikt met fp16=False. De resulterende transcriptie wordt vervolgens opgeslagen in een bestand genaamd 'scan.txt' en geretourneerd als tekst.

De functie run_text_prompt neemt een message en een chat_history als invoer. Het gebruikt de phi_demo functie om een reactie van een chatbot te genereren op basis van het ingevoerde bericht. De gegenereerde reactie wordt vervolgens doorgegeven aan de talk functie, die de reactie omzet in een audiobestand en het bestandspad retourneert. De Audio-klasse wordt gebruikt om het audiobestand weer te geven en af te spelen. De audio wordt weergegeven met behulp van de display functie uit de IPython.display module, en het Audio-object wordt gemaakt met de parameter autoplay=True, zodat de audio automatisch begint te spelen. De chat_history wordt bijgewerkt met het ingevoerde bericht en de gegenereerde reactie, en een lege string en de bijgewerkte chat_history worden geretourneerd.

De str-klasse is een ingebouwde klasse in Python die een reeks tekens vertegenwoordigt. Het biedt verschillende methoden voor het manipuleren en werken met strings, zoals capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, en meer. Deze methoden stellen je in staat om bewerkingen uit te voeren zoals zoeken, vervangen, formatteren en manipuleren van strings.

De Audio-klasse is een aangepaste klasse die een audio-object vertegenwoordigt. Het wordt gebruikt om een audiospeler te maken in de Jupyter Notebook-omgeving. De klasse accepteert verschillende parameters zoals data, filename, url, embed, rate, autoplay, en normalize. De data-parameter kan een numpy-array zijn, een lijst van samples, een string die een bestandsnaam of URL vertegenwoordigt, of ruwe PCM-data. De filename-parameter wordt gebruikt om een lokaal bestand op te geven om de audiodata uit te laden, en de url-parameter wordt gebruikt om een URL op te geven om de audiodata te downloaden. De embed-parameter bepaalt of de audiodata moet worden ingesloten met behulp van een data-URI of moet worden verwezen vanuit de oorspronkelijke bron. De rate-parameter specificeert de bemonsteringssnelheid van de audiodata. De autoplay-parameter bepaalt of de audio automatisch moet beginnen met afspelen. De normalize-parameter specificeert of de audiodata moet worden genormaliseerd (herschaald) naar het maximale mogelijke bereik. De Audio-klasse biedt ook methoden zoals reload om de audiodata opnieuw te laden vanuit een bestand of URL, en attributen zoals src_attr, autoplay_attr, en element_id_attr om de bijbehorende attributen voor het audio-element in HTML op te halen.

Over het algemeen worden deze functies en klassen gebruikt om audio naar tekst te transcriberen, audio-reacties van een chatbot te genereren, en audio weer te geven en af te spelen in de Jupyter Notebook-omgeving.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we ons best doen voor nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
